# reconstruct the model for Table 2  1993 QS fe_13 data and predict the line intensities

In [ ]:
import json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import ChiantiPy.tools.util as chutil
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm

In [ ]:
%autoreload 3

In [ ]:
matplotlib qt

In [ ]:
pwd

In [ ]:
myIon = 'fe_13'

In [ ]:
nameDict = chutil.convertName(myIon)

In [ ]:
nameDict.keys()

In [ ]:
nameDict['spectroscopic']

In [ ]:
nameDict['experimental']

In [ ]:
ls

## the fe_13 intensities are in the .json file

In [ ]:
with open('tab2_1993_qs_fe_13.json', 'r') as inpt:
    specData = json.load(inpt)

In [ ]:
specData['ref']

In [ ]:
specData.keys()

In [ ]:
wghtFactor = 0.2

In [ ]:
box = mm.maker(specData, wghtFactor = wghtFactor, verbose = True)

In [ ]:
ls

## open the pickled match file

In [ ]:
matchName = 'tab2_1993_qs_fe_13_match.pkl'

In [ ]:
with open(matchName,'rb') as inpt:
    match = pickle.load(inpt)

In [ ]:
match.keys()

## temperature and density are in the pickled match file

In [ ]:
match['EDensity'].shape

In [ ]:
match['EDensity'][0]

In [ ]:
dens = match['EDensity']

In [ ]:
' density range = %10.2e to %10.2e'%(dens.min(), dens.max())

## load the matches saved in the pickle file

In [ ]:
box.loadMatch(matchName)

## demo #1  gives a prettier plot

In [ ]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 3.e+26, myIon, horizontalalignment='center', fontsize=16)
plt.xlabel('Electron Density (cm$^{-3}$)', fontsize=14)
plt.ylabel('Emissin Measure (cm$^{-5}$)', fontsize=14)
plt.tight_layout()

In [ ]:
print(' # of density values %5i'%(box.EDensity.size))

## from the plot, a density of 7.e+8  and an em of 1.e+27 is estimated

In [ ]:
Dindex = np.argmin(np.abs(box.EDensity - 7.e+8))
' Dindex = %5i'%(Dindex)

In [ ]:
Dindex.size

In [ ]:
box.emSetIndices(Dindex, verbose=1)
print('density set to %10.2e'%(box.EDensity[Dindex]))

In [ ]:
em = 1.e+27
emLog = np.log10(em)
' em  %10.2e  emLog %10.2f'%(em, emLog)

In [ ]:
box.emSet(emLog)

## can reset the WghtFactor 

In [ ]:
box.WghtFactor

In [ ]:
box.WghtFactor = 0.2  # or whatever is appropriate

## predict the line intensities from the temperature indices and log_10 emission measures

In [ ]:
box.predict()

In [ ]:
box.predictPrint(filename='table2-1993-QS_fe_13_prior_predict_print.txt', sort='wvl')

In [ ]:
box.Nparams

In [ ]:
box.diffPrint(filename='table2-1993-QS_fe_13_prior_diff_print.txt', sort='wvl')

## predictPrint outputs to  'filename' 

## 'sort' can be set to 'wvl' or 'ion'

In [ ]:
box.Diff.keys()

In [ ]:
wvl = box.Diff['wvl']
diff = box.Diff['diffOverInt']

In [ ]:
diffMean = diff.mean()
diffStd = diff.std()

In [ ]:
' diffMean %10.3f  diff Std  %10.3f'%(diffMean, diffStd)

In [ ]:
plt.figure()
plt.plot(wvl, diff,'o')

In [ ]:
plt.axhline(diffMean, color='k', lw=2, label='Mean')
plt.axhline(diffMean + diffStd, color='r', lw=2, linestyle='--', label='1 std')
plt.axhline(diffMean - diffStd, color='r', lw=2, linestyle='--')  #, label='1 std')
plt.axhline(diffMean + 2.*diffStd, color='b', lw=2, linestyle='dotted', label='2 std')
plt.axhline(diffMean - 2.*diffStd, color='b', lw=2, linestyle='dotted')  #, label='2 std')
plt.axhline(diffMean + 3.*diffStd, color='g', lw=2, linestyle='dotted', label='3 std')
plt.axhline(diffMean - 3.*diffStd, color='g', lw=2, linestyle='dotted')  #, label='3 std')

In [ ]:
plt.xlabel('Wavelength ($\AA$)', fontsize=14)
plt.ylabel(r'(Obs - Pred)/(w $\times$ Obs)', fontsize=14)

In [ ]:
plt.legend(loc='upper right', bbox_to_anchor=(0.99, 1.0), fontsize=12)

In [ ]:
plt.tight_layout()

## predictPrint povides a way to see if the matches are more or less correct

## next, open the demo #3 notebooks to do a brute-forch chi-squared search